In [1]:
import os
# os.getcwd()
os.chdir("C:/Users/Anthony.Lighterness/Work/.iWork/Project_Work/Git_Repos/data_quality_backend/DQMaRC-dev/DQMaRC")

In [1]:
import pandas as pd
import numpy as np

# Import DQMaRC
from DataQuality import DataQuality

# Import utilities for data quality visualisation
from UtilitiesDQMaRC import (
    MetricCalculator,
    BarPlotGenerator,
    DonutChartGenerator,
    overall_quality_fx, col_good, col_bad)

ModuleNotFoundError: No module named 'DataQuality'

In [3]:
from ipydatagrid import DataGrid, TextRenderer

# Read in example data as pandas dataframe
df = pd.read_csv('./data/toydf_subset.csv')
DataGrid(
    df,
    editable=False,
    header_visibility='column',
    base_column_size = 180,
    base_column_header_size = 30,
    header_renderer=TextRenderer(background_color='lightblue'),
    grid_style={"header_background_color": "lightblue"}
    )

DataGrid(auto_fit_params={'area': 'all', 'padding': 30, 'numCols': None}, base_column_header_size=30, base_col…

In [4]:
# Initialise a DQ object by passing your data to the tool
dq = DataQuality(df)

# Retrieve default test parameter form the object. We will edit this in the next step
test_params = dq.get_test_params()

# View the test parameters template
DataGrid(
    test_params,
    editable=True,
    header_visibility='column',
    base_column_size = 180,
    base_column_header_size = 30,
    header_renderer=TextRenderer(background_color='#fb9a99'),
    grid_style={"header_background_color": "#fb9a99"}
    )

Warning - Using default parameters. For best results edit parameters and reload using set_test_params() method


DataGrid(auto_fit_params={'area': 'all', 'padding': 30, 'numCols': None}, base_column_header_size=30, base_col…

In [5]:
# Read in example data as pandas dataframe
test_params_definitions = pd.read_csv('./data/test_params_definitions.csv')

DataGrid(
    test_params_definitions,
    editable=False,
    header_visibility='column',
    base_column_size = 180,
    base_column_header_size = 30,
    header_renderer=TextRenderer(background_color='#b2df8a'),
    grid_style={"header_background_color": "#b2df8a"}
    )

DataGrid(auto_fit_params={'area': 'all', 'padding': 30, 'numCols': None}, base_column_header_size=30, base_col…

In [6]:
# Datetime format
test_params.loc[test_params['Field']=='Date_of_Diagnosis', 'Date_Format'] = "%d/%m/%Y"
test_params.loc[test_params['Field']=='Date_of_Birth', 'Date_Format'] = "%d/%m/%Y"
test_params.loc[test_params['Field']=='Datetime_Event1', 'Date_Format'] = "%d/%m/%Y %H:%M"
test_params.loc[test_params['Field']=='Datetime_Logging1', 'Date_Format'] = "%d/%m/%Y %H:%M"

# Another way to do this:
# test_params.at[3,'Date_Format']="%d/%m/%Y" # Date of Diagnosis
# test_params.at[4,'Date_Format']="%d/%m/%Y" # Date of Birth
# test_params.at[15,'Date_Format']="%d/%m/%Y %H:%M" # Datetime_Event1
# test_params.at[16,'Date_Format']="%d/%m/%Y %H:%M" # Datetime_Logging1

In [7]:
# Completeness
# Set to true for all fields
test_params['Completeness_NULL'] = True # Default value
test_params['Completeness_Empty'] = True # Default value

# Set Completeness Encoding only for the "Gender" column
test_params.at[2,'Completeness_Encoded']=True
test_params.at[2,'Completeness_Encoded_Mapping']='Unknown|Not Known|na' # use pipes for multiple possible codes for missingness

C:\Users\Anthony.Lighterness\AppData\Local\Temp\ipykernel_2364\3228092927.py:8: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Unknown|Not Known|na' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  test_params.at[2,'Completeness_Encoded_Mapping']='Unknown|Not Known|na' # use pipes for multiple possible codes for missingness


In [8]:
# Uniqueness
# Set to true for all fields
test_params['Uniqueness_rows'] = True # Default value

In [9]:
# Consistency
# Set up consistency checks between the "Metastatic_Indicator" and "Tumour_M_Stage" columns
test_params.loc[test_params['Field']=='Metastatic_Indicator', 'Consistency_Compare'] = True
test_params.loc[test_params['Field']=='Metastatic_Indicator', 'Consistency_Compare_Field'] = 'Tumour_M_Stage'
test_params.loc[test_params['Field']=='Metastatic_Indicator', 'Consistency_Compare_Mapping'] = '{["Absent"]: ["M0"], ["Present"]: ["M1", "M1a"]}'

# Set up consistency checks between date fields "Date_of_Birth" and "Date_of_Diagnosis"
test_params.loc[test_params['Field']=='Date_of_Birth', 'Consistency_Date_Relations'] = True
test_params.loc[test_params['Field']=='Date_of_Birth', 'Consistency_Date_Relations_Field'] = 'Date_of_Diagnosis'
test_params.loc[test_params['Field']=='Date_of_Birth', 'Consistency_Date_Relationship'] = '>' # i.e. raise an error if Date of Birth > Date of Diagnosis

C:\Users\Anthony.Lighterness\AppData\Local\Temp\ipykernel_2364\2999151425.py:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Tumour_M_Stage' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  test_params.loc[test_params['Field']=='Metastatic_Indicator', 'Consistency_Compare_Field'] = 'Tumour_M_Stage'
C:\Users\Anthony.Lighterness\AppData\Local\Temp\ipykernel_2364\2999151425.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '{["Absent"]: ["M0"], ["Present"]: ["M1", "M1a"]}' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  test_params.loc[test_params['Field']=='Metastatic_Indicator', 'Consistency_Compare_Mapping'] = '{["Absent"]: ["M0"], ["Present"]: ["M1", "M1a"]}'
C:\Users\Anthony.Lighterness\AppData\Local\Temp\ipykernel_2364\2999151425.py:9

In [10]:
# Timeliness
# Raise an error if a time difference threshold of 10 minutes is exceeded
test_params.loc[test_params['Field']=='Datetime_Event1', 'Timeliness_Date_Diff'] = True
test_params.loc[test_params['Field']=='Datetime_Event1', 'Timeliness_Date_Diff_Field'] = 'Datetime_Logging1'
test_params.loc[test_params['Field']=='Datetime_Event1', 'Timeliness_Date_Diff_Threshold'] = '10' # i.e. raise an error if timediff >10 minutes

C:\Users\Anthony.Lighterness\AppData\Local\Temp\ipykernel_2364\4144378692.py:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Datetime_Logging1' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  test_params.loc[test_params['Field']=='Datetime_Event1', 'Timeliness_Date_Diff_Field'] = 'Datetime_Logging1'
C:\Users\Anthony.Lighterness\AppData\Local\Temp\ipykernel_2364\4144378692.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '10' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  test_params.loc[test_params['Field']=='Datetime_Event1', 'Timeliness_Date_Diff_Threshold'] = '10' # i.e. raise an error if timediff >10 minutes


In [11]:
# Validity
# (1) Future Dates
test_params.loc[test_params['Field']=='Date_of_Diagnosis', 'Validity_Dates_Future'] = True
test_params.loc[test_params['Field']=='Date_of_Birth', 'Validity_Dates_Future'] = True

# (2) Date Outliers
test_params.loc[test_params['Field']=='Date_of_Diagnosis', 'Validity_Date_Range'] = True
test_params.loc[test_params['Field']=='Date_of_Birth', 'Validity_Date_Range'] = True

test_params.loc[test_params['Field']=='Date_of_Diagnosis', 'Validity_Date_Range_Min'] = '2011-01-01'
test_params.loc[test_params['Field']=='Date_of_Birth', 'Validity_Date_Range_Min'] = '1900-01-01'

test_params.loc[test_params['Field']=='Date_of_Diagnosis', 'Validity_Date_Range_Max'] = '2023-07-07'
test_params.loc[test_params['Field']=='Date_of_Birth', 'Validity_Date_Range_Max'] = '2023-01-01'

# (3) Numerical Outliers
test_params.loc[test_params['Field']=='Age', 'Validity_Range'] = True
test_params.loc[test_params['Field']=='Age', 'Validity_Range_Min'] = 0
test_params.loc[test_params['Field']=='Age', 'Validity_Range_Max'] = 120

test_params.loc[test_params['Field']=='Height_cm', 'Validity_Range'] = True
test_params.loc[test_params['Field']=='Height_cm', 'Validity_Range_Min'] = 20
test_params.loc[test_params['Field']=='Height_cm', 'Validity_Range_Max'] = 210

# (4) NHS Number Validator
test_params.loc[test_params['Field']=='NHS_Number', 'Validity_NHS_Number'] = True

# (5) UK Postcode Validator
test_params.loc[test_params['Field']=='Postcode', 'Validity_Postcode_UK'] = True

# (6) Data Standards
# Either create and save your own data standard
#lu_filename_user_made = './data/lookups/LU_toydf_gender_user_made.csv'
lu_filename_user_made = 'LU_toydf_gender_user_made.csv'
# dq.save_user_lookup(pd.DataFrame({'Code': ['Male', 'Female']}), lu_filename_user_made)

# Or access a pre-defined list saved as a csv file
lu_filename = '..src/data/lookups/LU_toydf_gender.csv'

# Here we will apply a pre-defined data standard for "gender"
test_params.loc[test_params['Field']=='Gender', 'Validity_Lookup_Table'] = True
test_params.loc[test_params['Field']=='Gender', 'Validity_Lookup_Table_Filename'] = lu_filename

# (7) Regular Expression Pattern
test_params.loc[test_params['Field']=='Datetime_Event1', 'Validity_Pattern'] = True
test_params.loc[test_params['Field']=='Datetime_Event1', 'Validity_Pattern_Regex'] = "(\d{2})/(\d{2})/(\d{4}) (\d{2}):(\d{2})"

test_params.loc[test_params['Field']=='Datetime_Event2', 'Validity_Pattern'] = True
test_params.loc[test_params['Field']=='Datetime_Event2', 'Validity_Pattern_Regex'] = "[0-9]{2}/[0-9]{2}/[0-9]{4} [0-9]{2}:[0-9]{2}"

C:\Users\Anthony.Lighterness\AppData\Local\Temp\ipykernel_2364\1136909361.py:10: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '2011-01-01' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  test_params.loc[test_params['Field']=='Date_of_Diagnosis', 'Validity_Date_Range_Min'] = '2011-01-01'
C:\Users\Anthony.Lighterness\AppData\Local\Temp\ipykernel_2364\1136909361.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '2023-07-07' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  test_params.loc[test_params['Field']=='Date_of_Diagnosis', 'Validity_Date_Range_Max'] = '2023-07-07'
C:\Users\Anthony.Lighterness\AppData\Local\Temp\ipykernel_2364\1136909361.py:42: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error 

In [12]:
# Accuracy
# Set a manually validated version of the data set as the gold standard
test_params['Gold_Standard'] = True

In [13]:
test_params_upload = pd.read_csv('./data/toydf_subset_test_params_24.05.16.csv')
test_params_upload
#df = pd.read_csv('../src/data/toydf_subset.csv')
DataGrid(
    test_params_upload,
    editable=False,
    header_visibility='column',
    base_column_size = 180,
    base_column_header_size = 30,
    header_renderer=TextRenderer(background_color='orange'),
    grid_style={"header_background_color": "orange"}
    )

DataGrid(auto_fit_params={'area': 'all', 'padding': 30, 'numCols': None}, base_column_header_size=30, base_col…

In [14]:
dq.set_test_params(test_params_upload)

# supply gold stand data - we are using the same dataset here for ease
dq.accuracy.set_gold_standard(df)

Using uploaded test parameters


In [15]:
dq.run_all_metrics()

# To run separately use following methods.
# dq.completeness.run_metrics()
# dq.uniqueness.run_metrics()
# dq.consistency.run_metrics()
# dq.timeliness.run_metrics()
# dq.validity.run_metrics()
# dq.accuracy.run_metrics()

format series not null: %d/%m/%Y
format series not null: %d/%m/%Y
format series not null: %d/%m/%Y
format series not null: %d/%m/%Y
format series not null: %d/%m/%Y %H:%M
format series not null: %d/%m/%Y %H:%M
format series not null: %d/%m/%Y %H:%M
format series not null: %d/%m/%Y %H:%M
format series not null: %d/%m/%Y %H:%M
format series not null: %d/%m/%Y %H:%M


C:\Users\Anthony.Lighterness\Work\.iWork\Project_Work\Git_Repos\data_quality_backend\DQMaRC-dev\DQMaRC\Consistency.py:135: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  return not (pd.to_datetime(x[col], format=date_format_field1) > pd.to_datetime(x[comp_col], format=date_format_field2))
C:\Users\Anthony.Lighterness\Work\.iWork\Project_Work\Git_Repos\data_quality_backend\DQMaRC-dev\DQMaRC\Consistency.py:145: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  return not (pd.to_datetime(x[col], format=date_format_field1) < pd.to_datetime(x[comp_col], format=date_format_field2))
C:\Users\Anthony.Lighterness\Work\.iWork\Project_Work\Git_Repos\data_quality_backend\DQMaRC-dev\DQMaRC\Consistency.py:145: UserWarning: Parsing dates in %d/%m/%Y %H:%M format when dayfirst

format series not null: %d/%m/%Y %H:%M
format series not null: %d/%m/%Y %H:%M
Error processing row: Unknown datetime string format, unable to parse: 1991/03/2024  00:00:00, at position 0
Error processing row: Unknown datetime string format, unable to parse: 1991/03/2024  00:00:00, at position 0
Error processing row: Unknown datetime string format, unable to parse: 1991/03/2024  00:00:00, at position 0
Error processing row: Unknown datetime string format, unable to parse: 1991/03/2024  00:00:00, at position 0
Error processing row: Unknown datetime string format, unable to parse: 1991/03/2024  00:00:00, at position 0
Error processing row: Unknown datetime string format, unable to parse: 1991/03/2024  00:00:00, at position 0
Error processing row: Unknown datetime string format, unable to parse: 1991/03/2024  00:00:00, at position 0
Error processing row: Unknown datetime string format, unable to parse: 1991/03/2024  00:00:00, at position 0
Error processing row: Unknown datetime string form

C:\Users\Anthony.Lighterness\Work\.iWork\Project_Work\Git_Repos\data_quality_backend\DQMaRC-dev\DQMaRC\Timeliness.py:76: UserWarning: Parsing dates in %d/%m/%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  start_time = pd.to_datetime(x[col], format=date_format_field1)
C:\Users\Anthony.Lighterness\Work\.iWork\Project_Work\Git_Repos\data_quality_backend\DQMaRC-dev\DQMaRC\Timeliness.py:77: UserWarning: Parsing dates in %d/%m/%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  end_time = pd.to_datetime(x[date_compare_col], format=date_format_field2)
C:\Users\Anthony.Lighterness\Work\.iWork\Project_Work\Git_Repos\data_quality_backend\DQMaRC-dev\DQMaRC\Timeliness.py:76: UserWarning: Parsing dates in %d/%m/%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warn

format series not null: %d/%m/%Y %H:%M
format series not null: %d/%m/%Y %H:%M
format series not null: %d/%m/%Y %H:%M
format series not null: %d/%m/%Y
Invalid input NHS Number: not a valid integer


C:\Users\Anthony.Lighterness\Work\.iWork\Project_Work\Git_Repos\data_quality_backend\DQMaRC-dev\DQMaRC\Validity.py:279: UserWarning: Parsing dates in %d/%m/%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  return pd.to_datetime(x, format=date_format_field1) > datetime.now()
C:\Users\Anthony.Lighterness\Work\.iWork\Project_Work\Git_Repos\data_quality_backend\DQMaRC-dev\DQMaRC\Validity.py:279: UserWarning: Parsing dates in %d/%m/%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  return pd.to_datetime(x, format=date_format_field1) > datetime.now()
C:\Users\Anthony.Lighterness\Work\.iWork\Project_Work\Git_Repos\data_quality_backend\DQMaRC-dev\DQMaRC\Validity.py:279: UserWarning: Parsing dates in %d/%m/%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warnin

In [16]:
raw = dq.raw_results()
agg = dq.aggregate_results()
row_agg = dq.aggregate_rows()

C:\Users\Anthony.Lighterness\Work\.iWork\Project_Work\Git_Repos\data_quality_backend\DQMaRC-dev\DQMaRC\DataQuality.py:213: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  row_agg = raw.groupby(cols, axis=1).sum()[sorted(set(cols), key=cols.index)]


In [17]:
DataGrid(
    agg,
    editable=False,
    header_visibility='column',
    base_column_size = 180,
    base_column_header_size = 30,
    header_renderer=TextRenderer(background_color='#fdbf6f'),
    grid_style={"header_background_color": "#fdbf6f"}
    )

DataGrid(auto_fit_params={'area': 'all', 'padding': 30, 'numCols': None}, base_column_header_size=30, base_col…

In [18]:
DataGrid(
    raw,
    editable=False,
    header_visibility='column',
    base_column_size = 180,
    base_column_header_size = 30,
    header_renderer=TextRenderer(background_color='#fb8072'),
    grid_style={"header_background_color": "#fb8072"}
    )

DataGrid(auto_fit_params={'area': 'all', 'padding': 30, 'numCols': None}, base_column_header_size=30, base_col…

In [19]:
df_DQ_full = df.join(raw, how="left")
DataGrid(
    df_DQ_full,
    editable=False,
    header_visibility='column',
    base_column_size = 180,
    base_column_header_size = 30,
    header_renderer=TextRenderer(background_color='#fb8072'),
    grid_style={"header_background_color": "#fb8072"}
    )

DataGrid(auto_fit_params={'area': 'all', 'padding': 30, 'numCols': None}, base_column_header_size=30, base_col…

### Completeness Example

In [20]:
gender_completeness_conditions = (df_DQ_full['Completeness_NULL_|_Gender']>0) | \
    (df_DQ_full['Completeness_Empty_|_Gender']>0) | \
        (df_DQ_full['Completeness_Encoded_|_Gender']>0)

df_DQ_full[['Gender','Completeness_NULL_|_Gender', 
            'Completeness_Empty_|_Gender', 'Completeness_Encoded_|_Gender', 
            'completeness_count_|_Gender']].loc[(gender_completeness_conditions)].head()


,Gender,Completeness_NULL_|_Gender,Completeness_Empty_|_Gender,Completeness_Encoded_|_Gender,completeness_count_|_Gender
2,.,0,0,1,1
7,.,0,0,1,1
10,NaN,1,0,0,1
11,NaN,1,0,0,1
15,.,0,0,1,1


### Uniqueness Examples

In [21]:
# Check which rows have duplication where they should be unique
df_DQ_full[['Patient_ID', 'NHS_number']][df_DQ_full[['Patient_ID', 'NHS_number']].duplicated()]


,Patient_ID,NHS_number
1,1,3417315905
11,10,6221980402


In [22]:
# Show how uniqueness flags are shown
# uniqueness_conditions = (df_DQ_full['row_uniqueness_|_full_row_uniqueness']>0)
uniqueness_conditions = (df_DQ_full['Patient_ID']==1) | (df_DQ_full['Patient_ID']==10)

df_DQ_full[['Patient_ID', 'NHS_number', 
            'row_uniqueness_|_full_row_uniqueness']].loc[(uniqueness_conditions)]


,Patient_ID,NHS_number,row_uniqueness_|_full_row_uniqueness
0,1,3417315905,1
1,1,3417315905,0
10,10,6221980402,1
11,10,6221980402,0


### Consistency Examples

In [23]:
consistency_conditions_metastatic_indicator = (df_DQ_full['Consistency_Compare_|_Metastatic_Indicator']>0)

df_DQ_full[['Tumour_M_Stage','Metastatic_Indicator', 
            'Consistency_Compare_|_Metastatic_Indicator', 
            'consistency_count_|_Metastatic_Indicator']].loc[(consistency_conditions_metastatic_indicator)].head(5)


,Tumour_M_Stage,Metastatic_Indicator,Consistency_Compare_|_Metastatic_Indicator,consistency_count_|_Metastatic_Indicator
0,M0,Present,1,1
1,M0,Present,1,1
2,M0,Present,1,1
3,M0,Present,1,1
4,M0,Present,1,1


In [24]:
consistency_conditions_dates = (df_DQ_full['Consistency_Date_Relations_|_Date_of_Diagnosis']>0) | \
    (df_DQ_full['Consistency_Date_Relations_|_Date_of_Birth']>0)

df_DQ_full[['Date_of_Birth','Date_of_Diagnosis',
            'Consistency_Date_Relations_|_Date_of_Birth',
            'Consistency_Date_Relations_|_Date_of_Diagnosis']].loc[(consistency_conditions_dates)].head(5)


,Date_of_Birth,Date_of_Diagnosis,Consistency_Date_Relations_|_Date_of_Birth,Consistency_Date_Relations_|_Date_of_Diagnosis
0,07/06/2090,23/04/2033,1,1
19,25/10/2015,12/07/2011,1,1
36,10/07/2017,31/05/2015,1,1
66,04/07/2014,26/01/2010,1,1
67,13/01/2020,07/12/2010,1,1


### Timeliness Examples

In [25]:
timeliness_conditions = (df_DQ_full['Timeliness_Date_Diff_|_Datetime_Event2']>0)

df_DQ_full[['Datetime_Event2','Datetime_Logging2',
            'Timeliness_Date_Diff_|_Datetime_Event2',
            'timeliness_count_|_Datetime_Event2']].loc[(timeliness_conditions)].head(5)


,Datetime_Event2,Datetime_Logging2,Timeliness_Date_Diff_|_Datetime_Event2,timeliness_count_|_Datetime_Event2
0,16/03/2024 00:00,16/03/2024 00:53,1,1
1,19/03/2024 00:00,19/03/2024 01:11,1,1
2,04/03/2024 00:00,04/03/2024 00:18,1,1
3,30/03/2024 00:00,30/03/2024 01:28,1,1
4,29/03/2024 00:00,29/03/2024 00:47,1,1


### Validity Examples - Future Dates

In [26]:
validity_future_dates_conditions = (df_DQ_full['Validity_Dates_Future_|_Date_of_Diagnosis']>0)

df_DQ_full[['Date_of_Diagnosis','Validity_Dates_Future_|_Date_of_Diagnosis']].loc[(validity_future_dates_conditions)].head(5)


,Date_of_Diagnosis,Validity_Dates_Future_|_Date_of_Diagnosis
0,23/04/2033,1


### Validity Examples - Outlier Dates

In [27]:
validity_outlier_dates_conditions = (df_DQ_full['Validity_Date_Range_|_Date_of_Diagnosis']>0)

df_DQ_full[['Date_of_Diagnosis','Validity_Date_Range_|_Date_of_Diagnosis']].loc[(validity_outlier_dates_conditions)].head(5)


,Date_of_Diagnosis,Validity_Date_Range_|_Date_of_Diagnosis
0,23/04/2033,1
4,17/09/2010,1
6,09/01/2013,1
7,19/06/2011,1
8,13/10/2012,1


### Validity Examples - NHS Numbers

In [28]:
validity_NHS_number_conditions = (df_DQ_full['Validity_NHS_Number_|_NHS_number']>0)

df_DQ_full[['NHS_number','Validity_NHS_Number_|_NHS_number']].loc[(validity_NHS_number_conditions)].head(5)


,NHS_number,Validity_NHS_Number_|_NHS_number
6,3563445,1
8,3463442,1
20,447659901,1
21,823933245,1
22,224961403,1


### Validity Examples - UK Postcodes

In [29]:
validity_UK_postcodes_conditions = (df_DQ_full['Validity_Postcode_UK_|_Postcode']>0)

df_DQ_full[['Postcode','Validity_Postcode_UK_|_Postcode']].loc[(validity_UK_postcodes_conditions)]


,Postcode,Validity_Postcode_UK_|_Postcode
6,AAA 1AA,1
8,BB1 1ABB,1


### Validity Examples - Invalid Codes

In [30]:
validity_codes_conditions = (df_DQ_full['Validity_Lookup_Table_|_ICD_10_Code']>0)

df_DQ_full[['ICD_10_Code','Validity_Lookup_Table_|_ICD_10_Code']].loc[(validity_codes_conditions)].head(10)


,ICD_10_Code,Validity_Lookup_Table_|_ICD_10_Code
2,Unkown,1
7,S72092Q,1
8,E10359,1
9,S42463K,1
10,S65911S,1
11,S65911S,1
12,X010XXD,1
14,S1315,1
17,S52353E,1
18,V403XXD,1


### Validity Examples - Numerical Ranges

In [31]:
validity_numerical_ranges_conditions = (df_DQ_full['Validity_Lookup_Table_|_ICD_10_Code']>0)

df_DQ_full[['ICD_10_Code','Validity_Lookup_Table_|_ICD_10_Code']].loc[(validity_numerical_ranges_conditions)].head(10)


,ICD_10_Code,Validity_Lookup_Table_|_ICD_10_Code
2,Unkown,1
7,S72092Q,1
8,E10359,1
9,S42463K,1
10,S65911S,1
11,S65911S,1
12,X010XXD,1
14,S1315,1
17,S52353E,1
18,V403XXD,1


### Validity Examples - Regex Patterns

In [32]:
validity_pattern_conditions = (df_DQ_full['Validity_Range_|_Height_cm']>0)

df_DQ_full[['Height_cm','Validity_Range_|_Height_cm']].loc[(validity_pattern_conditions)].head(10)


,Height_cm,Validity_Range_|_Height_cm
0,220.50,1
4,7.48,1
8,296.97,1
10,210.99,1
12,289.45,1
16,202.02,1
17,227.60,1
19,248.42,1
21,297.19,1
23,249.22,1


In [33]:
#df_DQ_full = df.join(raw, how="left")
# source_df_raw = df.join(raw)

# Prepare DQ Dashboard
raw_subset = raw.filter(regex='completeness|validity|consistency|uniqueness_count|accuracy|timeliness')
calculator = MetricCalculator(raw_subset)

# Simulate the calculation step, calculate aggregates
calculator.calculate_metrics()

summary_results = calculator.result
summary_results['Colour_Good'] = summary_results.apply(col_good, axis=1)
summary_results['Colour_Bad'] = summary_results.apply(col_bad, axis=1)
summary_results['Colour_NA'] = '#B2C3C6'

# Overall quality label
from IPython.display import HTML

# Function to display overall quality in a Jupyter Notebook
def display_overall_quality_label():
    if not summary_results.empty:
        data1 = summary_results[summary_results['Prop_NA'] == 0]
        avg_prop_good = data1['Prop_Good'].mean()
        overall_quality_level, background_color, text_colour = overall_quality_fx(avg_prop_good)

        overall_quality_text = f"Overall Quality: {overall_quality_level}"
        html = f"""
        <div style="
            background-color: {background_color};
            padding: 10px;
            border-radius: 5px;
            color: {text_colour};
            border: 2px solid {text_colour};
            text-align: center;
            width: 300px;">
            {overall_quality_text}
        </div>
        """
        return HTML(html)
    else:
        return HTML("<div style='text-align: center;'>No data available</div>")

# Use the function to display the result
display_overall_quality_label()

In [34]:
DonutChartGenerator(summary_results).plot_donut_charts()

In [35]:
BarPlotGenerator(summary_results, "completeness").plot_bar()

In [36]:
BarPlotGenerator(summary_results, "consistency").plot_bar()

In [37]:
BarPlotGenerator(summary_results, "timeliness").plot_bar()

In [38]:
BarPlotGenerator(summary_results, "uniqueness").plot_bar()

In [39]:
BarPlotGenerator(summary_results, "validity").plot_bar()